In [1]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so

In [2]:
import numpy as np

In [3]:
input_path = '../../data/lilium_highres.json'
target_mesh_path = '../../data/target_meshes/lilium_highres.obj'
# input_path = '../../data/TB.json'
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
# input_data['arm_plate_edge_offset'] /= 2
width = 6
# width = 2*input_data['arm_plate_edge_offset']
thickness = width * 0.5 # 1.5 mm # FIX from mm to meters everywhere
cross_section = [thickness, width]
# print(len(input_data['scale_factors']), len(input_data['base_mesh_f']))
# base_mesh = mesh.Mesh(input_data['base_mesh_v'], input_data['base_mesh_f'])
# mesh.save('../../data/hemisphere_base.msh', base_mesh.vertices(), base_mesh.elements())
# print(input_data['scale_factors'])

KeyError: 'bbox_diagonal'

### Initialization

In [4]:
# curr_um = umbrella_mesh.UmbrellaMesh(io)
curr_um = umbrella_mesh.UmbrellaMesh(target_mesh_path, io)


In [5]:
curr_um.setMaterial(elastic_rods.RodMaterial('rectangle', 1400, 0.35, cross_section, stiffAxis=elastic_rods.StiffAxis.D1), 
                    elastic_rods.RodMaterial('rectangle', 14000, 0.35, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))

In [6]:
curr_um.energy(UmbrellaEnergyType.Full)

3.86380536437662e-19

### Pin Rigid Motion



In [7]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
# fixedVars = list(range(jdo, jdo + 6)) + curr_um.rigidJointAngleDoFIndices()
fixedVars = curr_um.rigidJointAngleDoFIndices()

### Equilibrium solve

In [8]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-5
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 1000
OPTS.verboseNonPosDef = False

In [9]:
rod_colors = []
for seg_id in range(curr_um.numSegments()):
    rod_colors.append(np.ones(curr_um.segment(seg_id).rod.numVertices())*seg_id)

In [10]:
import mesh
from visualization_helper import *
target_mesh = mesh.Mesh(target_mesh_path)
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
# view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)

view.show()

Renderer(camera=PerspectiveCamera(aspect=1.7066666666666668, children=(PointLight(color='#999999', position=(0…

In [11]:
dof = curr_um.getDoFs()
for i in range(curr_um.numJoints()):
    if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.X):
        dof[curr_um.dofOffsetForJoint(i) + 6] = 1e-3
curr_um.setDoFs(dof)

In [12]:
view.showScalarField(rod_colors)

In [13]:
# def eqm_callback(prob, i):
#     if (i % 20 == 0):
#         view2.showScalarField(rod_colors)

In [14]:
curr_um.uniformDeploymentEnergyWeight = 0.1
curr_um.targetDeploymentHeight = thickness
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 10000
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(0.5)

In [15]:
curr_um.energyElastic(), curr_um.energyDeployment(), curr_um.energyRepulsion(), curr_um.energyAttraction()

(2.347372359320351, 46269.67965069007, 0.0, 700.3652522656257)

In [16]:
import benchmark
benchmark.reset()
report = umbrella_mesh.compute_equilibrium(curr_um, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
benchmark.report()
view.showScalarField(rod_colors)

0	46972.4	2350.6	2350.6	0.000488281	0
1	46972	2473.62	2473.62	0.0325849	1
Added constraint 3320 to working set
Added constraint 3328 to working set
Added constraint 3330 to working set
Added constraint 3342 to working set
Added constraint 3344 to working set
Added constraint 3347 to working set
Added constraint 3349 to working set
Added constraint 3352 to working set
Added constraint 3355 to working set
Added constraint 3360 to working set
Added constraint 3362 to working set
Added constraint 3374 to working set
Added constraint 3376 to working set
Added constraint 3388 to working set
Added constraint 3390 to working set
Added constraint 3402 to working set
Added constraint 3404 to working set
Added constraint 3452 to working set
Added constraint 3454 to working set
Added constraint 3457 to working set
Added constraint 3460 to working set
Added constraint 3462 to working set
Added constraint 3569 to working set
Added constraint 3571 to working set
Added constraint 3573 to working set
A

In [17]:
1.37108 / 728

0.0018833516483516485

In [18]:
1.53871/728

0.0021136126373626375

In [19]:
report.success

True

In [20]:
curr_um.uniformDeploymentEnergyWeight = 100
curr_um.attractionWeight = 1

In [21]:
benchmark.reset()
report = umbrella_mesh.compute_equilibrium(curr_um, options = OPTS, fixedVars = fixedVars)
benchmark.report()
view.showScalarField(rod_colors)

0	97645.3	6242.61	6242.61	0	1
Added constraint 4318 to working set
Added constraint 4322 to working set
Added constraint 4324 to working set
Added constraint 4325 to working set
Added constraint 4327 to working set
Added constraint 4331 to working set
Added constraint 4334 to working set
Added constraint 4336 to working set
Added constraint 4339 to working set
Added constraint 4341 to working set
Added constraint 4343 to working set
Added constraint 4344 to working set
Added constraint 4346 to working set
Added constraint 4348 to working set
Added constraint 4349 to working set
Added constraint 4355 to working set
Added constraint 4356 to working set
Added constraint 4360 to working set
Added constraint 4363 to working set
Added constraint 4364 to working set
Added constraint 4368 to working set
Added constraint 4370 to working set
Added constraint 4374 to working set
Added constraint 4377 to working set
Added constraint 4378 to working set
Added constraint 4380 to working set
Added co

In [22]:
report.success

True